# Movie Genre Classification

# IMAGES Project Deep Learning

Integrantes:
Angela María Arias Rojas 201728551 - Raúl Andrés Pardo Moreno 201727367

# KAGGLE TEAM: RA

Input: Movie Images

Output: Probability of the movie belong to each genre.

# PART 1. Machine Learning Algorithm

# i. Reading Data

Firstly, we upload the python modules and training and test text data from the available files:

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm
import os
%matplotlib inline

In [2]:
#Personal path where the files are uploaded from.
path='C:/AAndres/Maestria Analytics/Semestre Vacaciones/Deep Learning/1 Class/AppliedDeepLearningClass-master/finalProject'

In [3]:
dataTraining = pd.read_csv(os.path.join(path, 'data', 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'data', 'dataTesting.csv'), encoding='UTF-8', index_col=0)


In [4]:
images_training = []
for i in tqdm(dataTraining.index):
    images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())

100%|██████████| 7895/7895 [01:53<00:00, 69.71it/s]


In [5]:
images_testing = []
for i in tqdm(dataTesting.index):
    images_testing.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())


100%|██████████| 3383/3383 [00:42<00:00, 80.10it/s]


In [6]:
images_training = np.stack(images_training)
images_training.shape, dataTraining.shape[0]

((7895, 40960), 7895)

In [7]:
images_testing = np.stack(images_testing)
images_testing.shape, dataTesting.shape[0]

((3383, 40960), 3383)

In [8]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [9]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


# ii. Applying PCA


With the objetive of evaluating various models and select the best of them, we reduce 'X' variable dimentionality in order to use it for running models in faster way.

In [10]:
pca = PCA(n_components=32)
images_training_pca = pca.fit_transform(images_training)

In [11]:
images_testing_pca = pca.transform(images_testing)

In [12]:
images_training_pca.shape, images_testing_pca.shape

((7895, 32), (3383, 32))

# iii. Setting up the variable 'Y'.

In [13]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])
y_genres.shape

(7895, 24)

# a) Random Forest Classifier Algorithm

First, we fit the Random Forest Classifier Algorithm to a sample of 4.000 images in order to see the performance of the model:

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

In [15]:
model_rf=OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=40, max_depth=4, random_state=88))

In [16]:
model_rf.fit(images_training[0:4000,:],y_genres[0:4000,:])

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
            oob_score=False, random_state=88, verbose=0, warm_start=False),
          n_jobs=1)

In [17]:
y_pred_rf = model_rf.predict_proba(images_training[4001:7000,:])

In [18]:
from sklearn.metrics import r2_score, roc_auc_score
roc_auc_score(y_genres[4001:7000,], y_pred_rf,average='macro')

0.6101483032382503

The score of the model is just 0.61 but using just 4.000 images. The parameters of the model were changed in order to find the combination that provides the best sample accuracy. 

Number of estimators was a sensitive parameter which provides a wide range of metrics on the sample. Finally, we found that nearly 40 parameters provided a the performance of the model. 

Now, we fit the model to the complet data and make predictions. The predictors to be used are the images without PCA.

In [19]:
model_rf.fit(images_training,y_genres)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_rf_img = model_rf.predict_proba(images_testing)

pd.DataFrame(y_pred_rf_img, index=dataTesting.index, columns=cols).to_csv('pred_fr_img100.csv', index_label='ID')

This model scored 0.5958 in kaggle, being our best model. For practical reasons, we run our model with gray images.

# PART 2. Deep Learning Algorithms

# a) CNN

Uploading modules to create the CNN model..

In [23]:
from keras import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from matplotlib import pyplot as plt


C:\Users\andres\.anaconda\navigator\Anaconda_\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Steps before, we uploaded and shaped X variable for a Random Forest model. Now, we need to reshape this images to use them in our CNN model:

In [30]:
images_training_rsh=images_training.reshape(images_training.shape[0],256,160,1)
images_testing_rsh=images_testing.reshape(images_testing.shape[0],256,160,1)
images_training_rsh=images_training_rsh.astype('float32')
images_testing_rsh=images_testing_rsh.astype('float32')
images_training_rsh/=255
images_testing_rsh/=255

In [31]:
images_training_rsh.shape, images_testing_rsh.shape

((7895, 256, 160, 1), (3383, 256, 160, 1))

# Setting up the CNN model: 

In [32]:
model=Sequential()
model.add(Convolution2D(32,3,3,activation='relu',input_shape=(256,160,1)))
model.add(Convolution2D(32,3,3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dense(24,activation='sigmoid'))


C:\Users\andres\.anaconda\navigator\Anaconda_\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(256, 160,...)`
  
C:\Users\andres\.anaconda\navigator\Anaconda_\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In order to find the best parameters for this model, we used a sample of 4.000 gray images (full size) and training it multiple  times changing parameters as activation, dropout and optimizers. 

According with those results, we noticed the best results of the model with relu activation, dropout less than 0.4 and with adam optimizer.

Some of the combinations were quite similar in the sample training data used, so we selected one of them.

Now, we fit the model with all images in gray using a batch size of 100 because we saw a peak in the accuracy around this number.

In [ ]:
model.fit(images_training_rsh,y_genres,batch_size=100,nb_epoch=5,verbose=1)

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_img_cnn = model.predict_proba(images_testing_rsh)

pd.DataFrame(y_pred_img_cnn, index=dataTesting.index, columns=cols).to_csv('pred_cnn_img102.csv', index_label='ID')

# b) Transfer Learning

For transfer learning, we have used the VGG16 Model. The 'imagenet' weigths are imported.  

In [36]:
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.models import Model
from keras.applications.vgg16 import VGG16 
model_vgg16 = VGG16(weights='imagenet', include_top=False)

import keras.optimizers


In order to get the best parameters combination, we tested multiple alternatives. Various activation layers and optimizers were included in previous models but the accuracy was quite similar and predictions were not good.

In [39]:
def deepTransferModel(input_size):
    x_input = Input(input_size)
    x = model_vgg16.get_layer("block5_pool")(x_input)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(24)(x)
    x = Activation("sigmoid")(x)
    model1 = Model(inputs=x_input, output=x)
    return model
model1 = deepTransferModel(images_training_rsh.shape[1:])
model1.compile(optimizer = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0), loss = "categorical_crossentropy", metrics = ["accuracy"])

C:\Users\andres\.anaconda\navigator\Anaconda_\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`
  # Remove the CWD from sys.path while we load stuff.


In [40]:
model1.fit(images_training_rsh, y_genres, epochs=5, verbose=1, batch_size=60)

Epoch 1/5
7895/7895 [==============================] - ETA: 18:47 - loss: 8.6042 - acc: 0.0000e+ - ETA: 17:28 - loss: 14.7945 - acc: 0.0583   - ETA: 16:45 - loss: 15.7337 - acc: 0.044 - ETA: 16:30 - loss: 17.2779 - acc: 0.062 - ETA: 16:13 - loss: 17.8342 - acc: 0.056 - ETA: 16:01 - loss: 18.5434 - acc: 0.063 - ETA: 15:49 - loss: 18.7984 - acc: 0.059 - ETA: 15:42 - loss: 19.2507 - acc: 0.064 - ETA: 15:28 - loss: 19.4766 - acc: 0.061 - ETA: 15:18 - loss: 19.1767 - acc: 0.061 - ETA: 15:06 - loss: 19.1973 - acc: 0.063 - ETA: 14:58 - loss: 19.2617 - acc: 0.063 - ETA: 14:48 - loss: 19.3575 - acc: 0.066 - ETA: 14:38 - loss: 19.1646 - acc: 0.065 - ETA: 14:32 - loss: 19.0611 - acc: 0.063 - ETA: 14:22 - loss: 18.9668 - acc: 0.063 - ETA: 14:15 - loss: 18.7817 - acc: 0.064 - ETA: 14:05 - loss: 18.9074 - acc: 0.063 - ETA: 14:00 - loss: 18.9414 - acc: 0.064 - ETA: 13:50 - loss: 19.0241 - acc: 0.064 - ETA: 13:42 - loss: 19.1388 - acc: 0.063 - ETA: 13:33 - loss: 18.9907 - acc: 0.065 - ETA: 13:26 - los

7895/7895 [==============================] - ETA: 14:25 - loss: 21.5519 - acc: 0.083 - ETA: 14:15 - loss: 20.5521 - acc: 0.116 - ETA: 14:18 - loss: 19.8705 - acc: 0.127 - ETA: 14:07 - loss: 19.9327 - acc: 0.112 - ETA: 14:04 - loss: 19.2715 - acc: 0.100 - ETA: 13:55 - loss: 19.4178 - acc: 0.097 - ETA: 13:45 - loss: 19.7354 - acc: 0.090 - ETA: 13:43 - loss: 19.8730 - acc: 0.079 - ETA: 13:35 - loss: 19.9003 - acc: 0.077 - ETA: 13:30 - loss: 19.7312 - acc: 0.080 - ETA: 13:22 - loss: 19.5629 - acc: 0.075 - ETA: 13:15 - loss: 19.7287 - acc: 0.077 - ETA: 13:08 - loss: 19.7771 - acc: 0.075 - ETA: 13:00 - loss: 19.9508 - acc: 0.078 - ETA: 12:55 - loss: 20.0456 - acc: 0.084 - ETA: 12:47 - loss: 19.8301 - acc: 0.081 - ETA: 12:41 - loss: 19.8629 - acc: 0.079 - ETA: 12:34 - loss: 20.1027 - acc: 0.082 - ETA: 12:27 - loss: 20.0690 - acc: 0.081 - ETA: 12:21 - loss: 20.1550 - acc: 0.080 - ETA: 12:14 - loss: 20.0141 - acc: 0.078 - ETA: 12:08 - loss: 20.0175 - acc: 0.078 - ETA: 12:02 - loss: 20.1011 - ac

7895/7895 [==============================] - ETA: 14:04 - loss: 19.6715 - acc: 0.116 - ETA: 14:10 - loss: 19.2983 - acc: 0.091 - ETA: 14:04 - loss: 19.1938 - acc: 0.077 - ETA: 14:03 - loss: 19.2909 - acc: 0.075 - ETA: 13:53 - loss: 18.9969 - acc: 0.083 - ETA: 13:45 - loss: 19.5819 - acc: 0.080 - ETA: 13:40 - loss: 19.5478 - acc: 0.078 - ETA: 13:32 - loss: 19.9252 - acc: 0.079 - ETA: 13:27 - loss: 19.6250 - acc: 0.083 - ETA: 13:19 - loss: 19.5550 - acc: 0.083 - ETA: 13:13 - loss: 19.5494 - acc: 0.077 - ETA: 13:09 - loss: 19.7088 - acc: 0.073 - ETA: 13:01 - loss: 19.7197 - acc: 0.080 - ETA: 12:56 - loss: 19.6075 - acc: 0.079 - ETA: 12:49 - loss: 19.6416 - acc: 0.080 - ETA: 12:43 - loss: 19.5670 - acc: 0.082 - ETA: 12:36 - loss: 19.3555 - acc: 0.083 - ETA: 12:30 - loss: 19.3730 - acc: 0.084 - ETA: 12:24 - loss: 19.3636 - acc: 0.082 - ETA: 12:17 - loss: 19.4820 - acc: 0.080 - ETA: 12:11 - loss: 19.6502 - acc: 0.085 - ETA: 12:04 - loss: 19.6552 - acc: 0.086 - ETA: 11:57 - loss: 19.6598 - ac

In [41]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_tl_img = model1.predict_proba(images_testing_rsh)

pd.DataFrame(y_pred_tl_img, index=dataTesting.index, columns=cols).to_csv('pred_tl_img01.csv', index_label='ID')

Transfer learning did not score well in kaggle. Possibly, the vgg16 weights have to been re-trained for this specific task in order to get better predictions.

# Conclusions:

Our best model was Random Forest. The CNN models required a huge computational capacity so it was difficult to create more complex models. For that reason, we have just used the gray images, lossing information but trying to use various alternatives.


The disadvantage of working with images is the dimensionality. Unfortunately, when dimentions were reduced drastically, the predictions were not optimal  